In [1]:
from tensorflow import keras
import tensorflow as tf
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
# load dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [3]:
def load_dataset():
    #load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    #encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

In [4]:
def normalize(train, test):
    # convert to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize- range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

In [6]:
pip install git+https://github.com/hyperopt/hyperopt.git

  Cloning https://github.com/hyperopt/hyperopt.git to /private/var/folders/v_/8s4s4snx4rn719x9kf1s6s080000gn/T/pip-req-build-vugymj2f
  Running command git clone -q https://github.com/hyperopt/hyperopt.git /private/var/folders/v_/8s4s4snx4rn719x9kf1s6s080000gn/T/pip-req-build-vugymj2f
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 204kB 9.1MB/s eta 0:00:01
  Created wheel for hyperopt: filename=hyperopt-0.2.7-py2.py3-none-any.whl size=966119 sha256=dc761b3419c67a0e61cadd97d8a1bf64a48bf870fdaf99603314e532d32293f0
  Stored in directory: /private/var/folders/v_/8s4s4snx4rn719x9kf1s6s080000gn/T/pip-ephem-wheel-cache-40vnx6tf/wheels/41/81/24/15fcba6ebbe35fc157f711f37249cbf4229bb3754ff607fd21
Successfully built hyperopt
Note: you may need to restart the kernel to use updated packages.


In [12]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np


def optimize_cnn(hyperparameter):
  # Define model using hyperparameters 
  cnn_model = Sequential([Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], activation='relu',
                                 input_shape=(32,32,3)), 
            MaxPooling2D(pool_size=(2,2)),
            Dropout(hyperparameter['dropout_prob']),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(hyperparameter['dropout_prob']), 
            Flatten(),
            Dense(32, activation='relu'), 
            Dense(10, activation='softmax'),])

    cnn_model.compile(optimizer=hyperparameter['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])

    cnn_model.fit(train_X, to_categorical(train_y), epochs=2, batch_size=256, verbose=0)
 
  performance = cnn_model.evaluate(valid_X, to_categorical(valid_y), verbose=0)

  print("Hyperparameters: ", hyperparameter, "Accuracy: ", performance[1])
  print("----------------------------------------------------")
  # We want to minimize loss i.e. negative of accuracy
  return({"status": STATUS_OK, "loss": -1*performance[1], "model":cnn_model})
  
train_X, train_y = trainX[:50000], trainY[:50000]
valid_X, valid_y = trainX[-10000:],trainY[-10000:]

# Define search space for hyper-parameters
space = {
    
    'conv_kernel_size': hp.choice('conv_kernel_size', [1, 3, 5]),

    'dropout_prob': hp.uniform('dropout_prob', 0.1, 0.35),

    'optimizer': hp.choice('optimizer', ['Adam', 'sgd']),
}

trials = Trials()
best = fmin(
        optimize_cnn,
        space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=10,
    )

import numpy as np
print("==================================")
print("Best Hyperparameters", best)

test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

performance = test_model.evaluate(trainX, to_categorical(trainY))

print("==================================")
print("Test Accuracy: ", performance[1])

Hyperparameters:                                      
{'conv_kernel_size': 3, 'dropout_prob': 0.15884064221945612, 'optimizer': 'Adam'}
Accuracy:                                             
0.10159999877214432                                   
----------------------------------------------------  
Hyperparameters:                                                                  
{'conv_kernel_size': 1, 'dropout_prob': 0.18858814375613153, 'optimizer': 'sgd'}  
Accuracy:                                                                         
0.10109999775886536                                                               
----------------------------------------------------                              
Hyperparameters:                                                                  
{'conv_kernel_size': 3, 'dropout_prob': 0.2954079963786884, 'optimizer': 'Adam'}  
Accuracy:                                                                         
0.10249999910593033               

NameError: name 'np' is not defined

In [14]:
import numpy as np
print("==================================")
print("Best Hyperparameters", best)

test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

performance = test_model.evaluate(trainX, to_categorical(trainY))

print("==================================")
print("Test Accuracy: ", performance[1])

Best Hyperparameters {'conv_kernel_size': 0, 'dropout_prob': 0.12806301031114153, 'optimizer': 1}
50000/50000 [==============================] - 27s 541us/step
Test Accuracy:  0.16553999483585358
